In [ ]:
"""
Embed tokens in the dataset
"""
# path to raw dataset
raw_dataset_path = './stlawu-webpages.jsonl'
print(raw_dataset_path)

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset('json', data_files=raw_dataset_path, split='train')
raw_dataset

raw_dataset['url'][:1000]


In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "intfloat/e5-mistral-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
# import torch

# device = torch.device("cuda")
# model.to(device)

In [ ]:
model.max_seq_length = 4096 * 2
print(model)
print(tokenizer)

In [ ]:

tokenized_text = tokenizer(raw_dataset[0]['html_doc'])
tokenized_text.input_ids[:10]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding='max_length', truncation=True, return_tensors="pt", max_length=model.max_seq_length
    )
    print(encoded_input)
    # encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    return model_output


In [ ]:
get_embeddings(raw_dataset['html_doc'][0]).last_hidden_state.shape
get_embeddings('Hi, this is me!').last_hidden_state.shape

In [ ]:
from html_cleaner import clean_html

In [ ]:
print(raw_dataset['html_doc'][0])

In [ ]:
print(clean_html(raw_dataset['html_doc'][0]))

In [ ]:
cleaned_html = clean_html(raw_dataset['html_doc'][20])

embeddings = get_embeddings(cleaned_html)
print(embeddings)


In [ ]:
print(embeddings.last_hidden_state.shape)
print(embeddings.last_hidden_state[0, :-3, :])

In [ ]:
from html_cleaner import clean_dataset_path
from datasets import load_from_disk
clean_dataset = load_from_disk(clean_dataset_path)

In [ ]:
print(clean_dataset)

In [ ]:
# getting rid off unknown stuff
clean_dataset = clean_dataset.filter(lambda x: '<div>' in x['html_doc'])

In [ ]:
clean_dataset[4421]

In [1]:
# adding faiss index
from create_embeddings import embeddings_dataset_path
from datasets import load_from_disk
import numpy as np

embeddings_dataset = load_from_disk(embeddings_dataset_path)

/home/nikolai/SearchEngine/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(len(embeddings_dataset['embeddings'][0]))

4096


In [ ]:
print(embeddings_dataset['embeddings'][0])
# print(type(embeddings_dataset['embeddings'][0]))

In [ ]:
embeddings_dataset.add_faiss_index(column='embeddings')

In [ ]:
print(embeddings_dataset)

In [ ]:
# saving it 
embeddings_dataset.save_to_disk(embeddings_dataset_path)